---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [1]:
import pandas as pd
import numpy as np

In [61]:
def answer_one():
    # the data is uploaded and prepared by removing unwanted rows
    energy = pd.read_excel('Energy Indicators.xls', skiprows=16, skipfooter=38)
    # using the '.drop' function, more data preparation was done by removing unwanted rows and columns
    energy.drop(energy.index[0], inplace=True)
    energy.drop(energy.columns[[0,1]], axis=1, inplace=True)
    # the names of each column header are replaced
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    # using the '.replace' function, values in the dataframe are replaced
    #energy.replace({'Energy Supply': {'...': np.nan},'Energy Supply per Capita': {'...': np.nan},'% Renewable': {'...': np.nan}}, inplace=True)
    energy.replace('...', np.nan, inplace=True)
       
    # using the '.str.replace', characters within parentheis and numbers were replaced with '' (nothing)
    energy['Country'] = energy['Country'].str.replace(r'\(.*\)','')
    energy['Country'] = energy['Country'].str.replace('\d+','')
    # using the '.str.strp', whitespaces were removed from 'Country' column
    energy['Country'] = energy['Country'].str.strip()
    
    # removed empty rows
    energy.dropna()
    
    # more data preparation done by changing some country names using the '.str.replace'
    energy['Country'] = energy['Country'].str.replace('Republic of Korea', 'South Korea')
    energy['Country'] = energy['Country'].str.replace('United States of America', 'United States')
    energy['Country'] = energy['Country'].str.replace('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom')
    energy['Country'] = energy['Country'].str.replace('China, Hong Kong Special Administrative Region', 'Hong Kong')
    
    energy['Energy Supply'] *= 1000000
    
    # the data is uploaded and prepared by removing unwanted rows
    GDP = pd.read_csv('world_bank.csv', skiprows=4)
    GDP = GDP.rename(columns = {'Country Name': 'Country'})
    
    # more data preparation done by changing some country names using the '.str.replace'
    GDP['Country'] = GDP['Country'].str.replace("Korea, Rep.", "South Korea")
    GDP['Country'] = GDP['Country'].str.replace("Iran, Islamic Rep.", "Iran",)
    GDP['Country'] = GDP['Country'].str.replace("Hong Kong SAR, China", "Hong Kong")
    GDP = GDP.drop(GDP.iloc[:,1:50], axis=1)
        
    ScimEn = pd.read_excel('scimagojr-3.xlsx')
    
    # merge the three tables and filter the data to the top 15 hughest ranking countries
    merge1 = pd.merge(ScimEn,energy, how='outer', left_on='Country', right_on='Country')
    merge2 = pd.merge(merge1, GDP, how='outer', left_on='Country', right_on='Country')
    merge2.set_index('Country', inplace=True)
    merge2.sort_values(['Rank'], inplace=True)
    merge2 = merge2[:15]
    
    return merge2

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [ ]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [98]:
def answer_two():
   # the data is uploaded and prepared by removing unwanted rows
    energy = pd.read_excel('Energy Indicators.xls', skiprows=16, skipfooter=38)
    # using the '.drop' function, more data preparation was done by removing unwanted rows and columns
    energy.drop(energy.index[0], inplace=True)
    energy.drop(energy.columns[[0,1]], axis=1, inplace=True)
    # the names of each column header are replaced
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    # using the '.replace' function, values in the dataframe are replaced
    #energy.replace({'Energy Supply': {'...': np.nan},'Energy Supply per Capita': {'...': np.nan},'% Renewable': {'...': np.nan}}, inplace=True)
    energy.replace('...', np.nan, inplace=True)
       
    # using the '.str.replace', characters within parentheis and numbers were replaced with '' (nothing)
    energy['Country'] = energy['Country'].str.replace(r'\(.*\)','')
    energy['Country'] = energy['Country'].str.replace('\d+','')
    # using the '.str.strp', whitespaces were removed from 'Country' column
    energy['Country'] = energy['Country'].str.strip()
    
    # removed empty rows
    energy.dropna()
    
    # more data preparation done by changing some country names using the '.str.replace'
    energy['Country'] = energy['Country'].str.replace('Republic of Korea', 'South Korea')
    energy['Country'] = energy['Country'].str.replace('United States of America', 'United States')
    energy['Country'] = energy['Country'].str.replace('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom')
    energy['Country'] = energy['Country'].str.replace('China, Hong Kong Special Administrative Region', 'Hong Kong')
    
    energy['Energy Supply'] *= 1000000
    
    # the data is uploaded and prepared by removing unwanted rows
    GDP = pd.read_csv('world_bank.csv', skiprows=4)
    GDP = GDP.rename(columns = {'Country Name': 'Country'})
    
    # more data preparation done by changing some country names using the '.str.replace'
    GDP['Country'] = GDP['Country'].str.replace("Korea, Rep.", "South Korea")
    GDP['Country'] = GDP['Country'].str.replace("Iran, Islamic Rep.", "Iran",)
    GDP['Country'] = GDP['Country'].str.replace("Hong Kong SAR, China", "Hong Kong")
    GDP = GDP.drop(GDP.iloc[:,1:50], axis=1)
    
    ScimEn = pd.read_excel('scimagojr-3.xlsx')
    
    # create a single dataframe where all the three sources are stacked and then all the unique 'Country' for three sources are taken
    country_unique = len(pd.concat([ScimEn['Country'],energy['Country'],GDP['Country']]).unique())
    
    # merge all three sources by 'Country' and the data is filtered with only the countries present in all sources remain
    num_merge = GDP.merge(energy, left_on='Country', right_on='Country', how='inner').merge(ScimEn, left_on='Country', right_on='Country', how='inner').shape[0]
    
    # take the difference to get the number of lost entries after merging the datasets
    lost_entries = country_unique - num_merge
    
    return lost_entries

<br>

Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [105]:
def answer_three():
    # call the answer for question #1
    Top15 = answer_one()
    # prepare the data by calling all the rows, and columns from 10 up to the last --> '.iloc[:,10:]'
    Top15 = Top15.iloc[:,10:]
    # using the '.mean' function, take the average per each row '(axis=1)' and sort them in descending order
    avgGDP = Top15.mean(axis=1).sort_values(ascending=False)
    return avgGDP

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [110]:
def answer_four():
    # call the answer for question #1
    Top15 = answer_one()
    # prepare the data by calling all the rows, and columns from 10 up to the last --> '.iloc[:,10:]'
    Top15 = Top15.iloc[:,10:]
    # using the '.mean' function, take the average per each row '(axis=1)' and sort them in descending order
    avgGDP = Top15.mean(axis=1).sort_values(ascending=False)
    # define the variable for the country with 6th highest GDP
    sixth = avgGDP.index[5]
    # the change in GDP for the country 'sixth' can be acquired by getting the difference of the GDP from the first and last columns
    diff = abs(Top15.loc[sixth].iloc[0] - Top15.loc[sixth].iloc[-1])
    return diff

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [115]:
def answer_five():
    # call the answer for question #1
    Top15 = answer_one()
    # create a series that contains only the 'Energy Supply per Capita' column and take the average
    ESpC_mean = Top15.iloc[:,8].mean()
    return ESpC_mean

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [126]:
def answer_six():
    # call the answer for question #1
    Top15 = answer_one()
    # retrieve the index (Country) with the highest value from column 9
    maxRen_country = Top15.iloc[:,9].idxmax()
    # retrive the highest value from column 9 
    maxRen_percent = Top15.iloc[:,9].max()
    # create a tuple for the country and the value
    return (maxRen_country, maxRen_percent)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [138]:
def answer_seven():
    # call the answer for question #1    
    Top15 = answer_one()
    # create a new column for the ratio of the self-citations and total citations
    Top15['Ratio'] = Top15['Self-citations'] / Top15['Citations']
    # retrieve the index (Coutry) with the highest ratio from the last column (-1)
    maxRatio_country = Top15.iloc[:,-1].idxmax()
    # retrive the highest value from the last column (-1)
    maxRatio = Top15.iloc[:,-1].max()
    # create a tuple for the country and the value
    return (maxRatio_country, maxRatio)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [142]:
def answer_eight():
    # call the answer for question #1
    Top15 = answer_one()
    # create a new column that estimates the total population for each country
    Top15['Population Estimate'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    # sort the values in descending order
    Top15.sort_values(['Population Estimate'], ascending=False, inplace=True)
    # retrieve the estimated third most populated country by calling the index 
    third_pop_est = Top15.index[2]
    return third_pop_est

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [147]:
def answer_nine():
    # call the answer for question #1
    Top15 = answer_one()
    # create a new column that estimates the total population for each country
    Top15['Population Estimate'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    # create a new column that estimates the citable documents per person
    Top15['Citable Docs per Capita'] = Top15['Citable documents'] / Top15['Population Estimate']
    # find the correlation between 'Citable Docs per Capita' and 'Energy Supply per Capita'
    corr_citabledDocs_energySupply = Top15['Citable Docs per Capita'].corr(Top15['Energy Supply per Capita'])
    return corr_citabledDocs_energySupply

In [ ]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [160]:
def answer_ten():
    # call the answer for question #1
    Top15 = answer_one()
    # create a column which stores a 1 if the '% Renewable' is greater the median of '% Renewable', and 0 if not
    Top15['HighRenew'] = np.where(Top15['% Renewable'] >= Top15['% Renewable'].median(), 1, 0)
    return Top15['HighRenew']

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [220]:
def answer_eleven():
    # call the answer for question #1
    Top15 = answer_one()
    
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    # create a new column that assigns the index 'Country' to its respective continent using the '.map' function
    Top15['Continent'] = Top15.index.map(ContinentDict)
    # create a new column that estimates the total population for each country
    Top15['Population Estimate'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    # using '.groupby', create a new dataframe with 'Continent' as the index and using '.agg', create columns with 'size', 'sum', 'mean', 'std' as aggregates
    continent_df = Top15.groupby('Continent')['Population Estimate'].agg({'size','sum','mean','std'})
    return continent_df

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [233]:
def answer_twelve():
    # call the answer for question #1
    Top15 = answer_one()
    
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    # create a new column that assigns the index 'Country' to its respective continent using the '.map' function
    Top15['Continent'] = Top15.index.map(ContinentDict)
    # using 'pd.cut', create a new column that assigns a country to a 'Bin' with respect to '% Renewable'
    Top15['Bin'] = pd.cut(Top15['% Renewable'], 5)
    # using '.groupyby', create a multiindex series of 'Continent' and 'Bin' and specify the size for each
    bins_continent_perRen = Top15.groupby(['Continent', 'Bin']).size()
    return bins_continent_perRen

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [245]:
def answer_thirteen():
    # call the answer for question #1
    Top15 = answer_one()
    # create a new column that estimates the total population for each country
    Top15['Population Estimate'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    # using '.apply', format the 'Population Estimate' to have thousand separators and using '.astype', convert the values from float to string
    return Top15['Population Estimate'].apply(lambda x: '{:,}'.format(x)).astype(str)

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!